In [174]:
%cd 20news-bydate\ 2\matlab/
import pandas as pd
import numpy as np
import time

[Errno 2] No such file or directory: '20news-bydate 2/matlab/'
/Users/phesami/Documents/DSE/phesami/DSE210/Week2/20news-bydate 2/matlab


In [427]:
def navive_bayes_train (data_DF, label_DF, map_DF, num_words, frequency_method):
    ####joing the label and the data####
    label_DF.index+=1
    data_label_DF=data_DF.join(label_DF, on='docIdx')

    ####creating a dummy DF which includes all words ID with count 1####
    dummy_missing_words=pd.DataFrame(np.zeros(num_words), columns=['dummy_count']).reset_index().rename(columns={'index':'wordIdx'})+1

    ####calculating the probability of each class. ie., P_j####
    total_docs=data_label_DF.groupby('docLabel')['docIdx'].count().sum()
    Pj=data_label_DF.groupby('docLabel')['docIdx'].count()/total_docs
    log_Pj=np.log(Pj)

    ####calculating the log of the probability of each word in each class. i.e., log(P_ji)
    word_count_Pj_DF=pd.DataFrame()
    for i in range(1,map_DF.shape[0]+1):

        ####joining the main dataframe with the dummy dataframe to include the missing words in each documents####
        word_count_DF=data_label_DF.ix[data_label_DF['docLabel']==i].merge(dummy_missing_words, on='wordIdx', how='right').sort('wordIdx')
        word_count_DF['count'].fillna(value=0, inplace=True)
        ####laplace smoothing is done at this step automatically as every word count is incremented by 1####
        ####adding the log(1+f) instead of f as an option for the last part of t
        if (frequency_method=='log'):
            word_count_DF['word_count_'+str(i)]=np.log(1+word_count_DF['count']+word_count_DF['dummy_count'])
        else:
            word_count_DF['word_count_'+str(i)]=word_count_DF['count']+word_count_DF['dummy_count']

        word_count_DF.drop(['count', 'dummy_count','docIdx', 'docLabel'], axis=1, inplace=True)
        word_count_Pj_temp_DF= word_count_DF.groupby('wordIdx').sum()
        word_count_Pj_temp_DF['log(P_'+str(i)+'i)']=np.log((word_count_Pj_temp_DF['word_count_'+str(i)])/(word_count_Pj_temp_DF['word_count_'+str(i)].sum()))
        word_count_Pj_DF=pd.concat([word_count_Pj_DF,word_count_Pj_temp_DF['log(P_'+str(i)+'i)']], axis=1)
    ####returns P_j and log(P_ji)####
    return word_count_Pj_DF, log_Pj

In [428]:
def naive_bayes_multinomial_classifier(data_DF, label_DF, word_count_Pj_DF, log_Pj):
    start = time.time()
    dummy_missing_words_test=pd.DataFrame(np.zeros(num_words), columns=['dummy_count']).reset_index().rename(columns={'index':'wordIdx'})
    dummy_missing_words_test['wordIdx']+=1
    dummy_missing_words_test.head()
    results={}
    for doc in data_DF['docIdx'].unique():
        data_train_count_DF=data_DF.ix[data_DF['docIdx']==doc].merge(dummy_missing_words_test, on='wordIdx', how='right').sort('docIdx')
        data_train_count_DF['count'].fillna(value=0, inplace=True)
        data_train_count_DF['docIdx'].fillna(value=doc, inplace=True)
        data_train_count_DF['word_count']=data_train_count_DF['count']+data_train_count_DF['dummy_count']
        word_count_sorted= data_train_count_DF.sort(columns='wordIdx')['word_count']
        word_count_rep=pd.concat([word_count_sorted]*20, axis=1)
        x_log=pd.DataFrame(word_count_rep.values*word_count_Pj_DF.values, columns=range(1,21),index=word_count_Pj_DF.index).sum()
        results[doc]= pd.DataFrame(x_log+log_Pj).idxmax()[0]

    error=np.mean(results.values()!=label_DF)*100
    end = time.time()
    print "error rate is:\n",error[0],'%'
    print "time it took on the data set:\n",int(end-start)/60, "minutes"

In [426]:
####Data importing and prepration####
data_DF=pd.read_csv("train.data", header=None, names=['docIdx','wordIdx','count'], sep=r"\s+")
map_DF=pd.read_csv("train.map", header=None, names=['category','categoryIdx'], sep=r"\s+")
label_DF=pd.read_csv("train.label", header=None, names=['docLabel'], sep=r"\s+")
num_words=pd.read_csv('vocabulary.txt').shape[0]

####training####
word_count_Pj_DF, log_Pj= navive_bayes_train (data_DF, label_DF, map_DF, num_words, 'normal')
word_count_Pj_DF.head()

,log(P_1i),log(P_2i),log(P_3i),log(P_4i),log(P_5i),log(P_6i),log(P_7i),log(P_8i),log(P_9i),log(P_10i),log(P_11i),log(P_12i),log(P_13i),log(P_14i),log(P_15i),log(P_16i),log(P_17i),log(P_18i),log(P_19i),log(P_20i)
wordIdx,,,,,,,,,,,,,,,,,,,,
1,-9.626822,-8.037989,-9.107572,-9.615915,-9.867290,-8.404160,-11.960255,-9.595849,-9.415573,-11.637980,-11.812874,-8.334283,-10.900279,-9.579829,-8.348430,-12.770284,-9.627332,-10.048035,-12.677129,-12.345487
2,-7.902074,-7.757374,-7.462416,-8.323147,-8.126824,-6.905562,-8.110108,-7.804090,-7.651985,-8.236782,-7.814674,-7.774667,-8.226131,-8.370869,-7.619178,-7.476979,-8.611412,-7.517872,-8.443022,-8.014753
3,-6.639458,-12.300669,-12.198615,-12.254972,-12.169875,-12.515034,-11.960255,-12.368438,-12.305945,-12.331127,-12.506022,-12.765100,-12.286574,-12.575562,-12.567938,-9.402988,-12.671855,-12.938407,-12.677129,-9.512273
4,-9.878137,-9.081793,-8.902778,-12.254972,-11.476728,-7.919914,-10.573961,-12.368438,-10.514186,-10.944833,-12.506022,-9.874728,-11.187961,-9.484519,-8.904376,-10.285377,-10.186948,-10.048035,-9.093610,-10.959192
5,-7.729702,-9.081793,-8.643267,-9.310533,-11.476728,-9.296158,-10.861643,-9.323916,-10.226504,-11.637980,-11.812874,-8.383073,-10.089349,-9.020214,-9.476895,-9.474447,-9.627332,-11.552113,-9.309833,-8.848979


In [429]:
####reloading data into new dataframe names to avoid confusion####
data_train_DF=pd.read_csv("train.data", header=None, names=['docIdx','wordIdx','count'], sep=r"\s+")
data_test_DF=pd.read_csv("test.data", header=None, names=['docIdx','wordIdx','count'], sep=r"\s+")
label_train_DF=pd.read_csv("train.label", header=None, names=['docLabel'], sep=r"\s+")
label_test_DF=pd.read_csv("test.label", header=None, names=['docLabel'], sep=r"\s+")

In [208]:
####classifying the train data####
naive_bayes_multinomial_classifier(data_train_DF, label_train_DF, word_count_Pj_DF, log_Pj) 

error rate is:
4.60555506256 %
time it took on the data set:
16 minutes


In [430]:
####classifying the test data####
naive_bayes_multinomial_classifier(data_test_DF, label_test_DF, word_count_Pj_DF, log_Pj) 

error rate is:
20.253164557 %
time it took on the data set:
9 minutes


### Results of the Naive Bayes Classifier ###
**1. Error rate on train set: 4.6%**

**2. Error rate on test set: 20.25%**

In [432]:
####part f-Optional###
####splitting the train data into 80% train set and 20% validation set####
####as our data and label are not the same size, we split label only and join the splitted label on data to derive splitted data####
from sklearn.cross_validation import train_test_split
label_train_split, label_train_val = train_test_split(label_train_DF, test_size=0.2)
label_train_split=label_train_split.sort()
label_train_val=label_train_val.sort()
data_train_DF_split = data_train_DF.join(label_train_split, on='docIdx', how='right').drop(['docLabel'], axis=1).reset_index(drop=True)
data_train_DF_val = data_train_DF.join(label_train_val, on='docIdx', how='right').drop(['docLabel'], axis=1).reset_index(drop=True)[:-1]

In [433]:
####training on the 80% of the date with replacing f with log(1+f) as word frequency####
word_count_Pj_DF_split, log_Pj_split = navive_bayes_train (data_train_DF_split, label_train_split, map_DF, num_words, 'log')
####measuring error with the 20% validation set with replacing f with log(1+f) as word frequency####
naive_bayes_multinomial_classifier(data_train_DF_split, label_train_split, word_count_Pj_DF_split, log_Pj_split)

error rate is:
8.07542983916 %
time it took on the data set:
13 minutes


In [431]:
####Naive Bayes results with log(1+f) instead of f###
word_count_Pj_DF, log_Pj= navive_bayes_train (data_DF, label_DF, map_DF, num_words, 'log')
naive_bayes_multinomial_classifier(data_test_DF, label_test_DF, word_count_Pj_DF, log_Pj) 

error rate is:
20.6662225183 %
time it took on the data set:
10 minutes


### Results of the Naive Bayes Classifier with $log(1+f)$ frequency###
**error rate on test set is 20.67%, slighlt higher than normal frequency count (i.e., $f$) method**

In [434]:
#TBD#
vocab_DF=pd.read_csv('vocabulary.txt')
####Removing stopwords####
from stop_words import get_stop_words
stop_words = get_stop_words('english')
####training on the 80% of the date with removing the stopwords####
#TBD
####measuring error with the 20% validation set with removing the stopwords####
#TBD